In [ ]:
#DDPG
!pip install torch==1.13.1 --extra-index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu


In [ ]:
!pip install numpy matplotlib

Parser

In [ ]:
class Task():
    def __init__(self,taskName,taskId, taskLength):
        # super().__init__(taskId, taskLength, 1, 0, 0, UtilizationModelFull(), UtilizationModelFull(), UtilizationModelFull()) #UtilizationModel??
        self.taskName=taskName
        self.taskId=taskId
        self.taskLength=taskLength
        self.allocated = False
        self.childList = []
        self.parentList = []
        self.fileList = []
        self.impact = 0.0
        self.taskFinishTime = -1.0
        self.type = None
        self.priority = 0
        self.depth = 0

    def setType(self, type):
        self.type = type

    def getType(self):
        return self.type

    def setPriority(self, priority):
        self.priority = priority

    def setDepth(self, depth):
        self.depth = depth

    def getPriority(self):
        return self.priority

    def getDepth(self):
        return self.depth

    def getChildList(self):
        return self.childList

    def setChildList(self, list):
        self.childList = list

    def setParentList(self, list):
        self.parentList = list

    def addChildList(self, list):
        self.childList.extend(list)

    def addParentList(self, list):
        self.parentList.extend(list)

    def getParentList(self):
        return self.parentList

    def addChild(self, task):
        self.childList.append(task)

    def addParent(self, task):
        self.parentList.append(task)

    def getFileList(self):
        return self.fileList

    def addFile(self, file):
        self.fileList.append(file)

    def setFileList(self, list):
        self.fileList = list

    def setImpact(self, impact):
        self.impact = impact

    def getImpact(self):
        return self.impact

    def setTaskFinishTime(self, time):
        self.taskFinishTime = time

    def getTaskFinishTime(self):
        return self.taskFinishTime

#depends on CloudLet
    def getProcessingCost(self):
        cost = self.getCostPerSec() * self.getActualCPUTime()
        file_size = sum(file.getSize() / Consts.MILLION for file in self.getFileList())
        cost += self.costPerBw * file_size
        return cost


In [ ]:
from enum import Enum
from typing import List

class FileType(Enum):
    NONE = 0
    INPUT = 1
    OUTPUT = 2

    def __new__(self, value):
        obj = object.__new__(self)
        obj._value_ = value
        return obj

class FileItem:
    def __init__(self, name, size):
        self.name = name
        self.size = size
        self.type = None

    def setName(self, name):
        self.name = name

    def setSize(self, size):
        self.size = size

    def setType(self, file_type):
        self.type = file_type

    def getName(self):
        return self.name

    def getSize(self):
        return self.size

    def getType(self):
        return self.type

    def is_real_input_file(self, file_list):
        if self.type == FileType.INPUT:
            for another in file_list:
                if another.get_name() == self.name and another.get_type() == FileType.OUTPUT:
                    return False
            return True
        return False

In [ ]:
class ReplicaCatalog:
    class FileSystem:
        SHARED = 'SHARED'
        LOCAL = 'LOCAL'

    fileName2File = {}
    fileSystem = None
    dataReplicaCatalog = {}


    def init(self, fs):
        self.fileSystem = fs
        self.dataReplicaCatalog = {}
        self.fileName2File = {}


    def getFileSystem(self):
        return self.fileSystem


    def getFile(self, fileName):
        return self.fileName2File.get(fileName)


    def setFile(self, fileName, file):
        self.fileName2File[fileName] = file


    def containsFile(self, fileName):
        return fileName in self.fileName2File


    def getStorageList(self, file):
        return self.dataReplicaCatalog.get(file, [])


    def addFileToStorage(self, file, storage):
        if file not in self.dataReplicaCatalog:
            self.dataReplicaCatalog[file] = []
        if storage not in self.dataReplicaCatalog[file]:
            self.dataReplicaCatalog[file].append(storage)

In [ ]:
import os
from enum import Enum
from xml.etree import ElementTree as ET



class WorkflowParser:
    def __init__(self, userId):
        self.userId = userId
        self.mName2Task = {}
        self.daxPath = "/content/drive/MyDrive/BTP2/Montage_100.xml"
        self.daxPaths = []
        self.jobIdStartsFrom = 1
        self.taskList = []

    def get_task_list(self):
        return self.taskList

    def set_task_list(self, task_list):
        self.taskList = task_list

    def parse(self):
        if self.daxPath:
            self.parse_xml_file(self.daxPath)
        elif self.daxPaths:
            for path in self.daxPaths:
                self.parse_xml_file(path)

    def set_depth(self, task, depth):
        if depth > task.getDepth():
            task.setDepth(depth)
        for cTask in task.getChildList():
            self.set_depth(cTask, task.getDepth() + 1)

    def parse_xml_file(self, path):
        try:
            tree = ET.parse(path)
            root = tree.getroot()
            for node in root:

              if node.tag == "{http://pegasus.isi.edu/schema/DAX}job":
                length = 0
                nodeName = node.get("id")
                nodeType = node.get("name")



                runtime = 0.1
                nodeTime = node.get("runtime")
                if nodeTime is not None:
                    runtime=float(nodeTime)
                    runtime = 1000*float(nodeTime)
                    if runtime < 100:
                        runtime = 100
                    length = int(runtime)
                else:
                    print(f"Cannot find runtime for {nodeName}, set it to be 0")

                # length *= Parameters.getRuntimeScale()

                mFileList = []
                for file in node.findall("uses"):
                    fileName = file.get("name")
                    if fileName is None:
                        fileName = file.get("file")
                    if fileName is None:
                        print("Error in parsing xml")

                    inout = file.get("link")
                    size = 0.0
                    fileSize = file.get("size")
                    if fileSize is not None:
                        size = float(fileSize)
                    else:
                        print(f"File Size not found for {fileName}")

                    if size == 0:
                        size = 1
                    type=FileType.NONE
                    if inout == "input":
                        type = FileType.INPUT
                    elif inout == "output":
                        type = FileType.OUTPUT
                    else:
                        print("Parsing Error")
                    tFile = None


                    if size < 0:
                       size = -size
                       print("Size is negative, assuming it is a parser error")


                    if type == FileType.OUTPUT:
                       tFile = FileItem(fileName, size)
                    elif ReplicaCatalog.containsFile(fileName):
                       tFile = ReplicaCatalog.getFile(fileName)
                    else:
                       tFile = FileItem(fileName, size)
                       ReplicaCatalog.setFile(fileName, tFile)

                    tFile.setType(type)
                    mFileList.append(tFile)



                task = Task(nodeName,self.jobIdStartsFrom, length)
                self.jobIdStartsFrom += 1
                task.setType(nodeType);
                # task.setUserId(self.userId);
                task.setFileList(mFileList)
                self.get_task_list().append(task)
                self.mName2Task[nodeName] = task
                # for file in mFileList:
                #    task.addRequired

              elif node.tag == "{http://pegasus.isi.edu/schema/DAX}child":
                child_name = node.get("ref")
                if child_name in self.mName2Task:
                    child_task = self.mName2Task[child_name]

                    for parent in node.findall("{http://pegasus.isi.edu/schema/DAX}parent"):
                        parent_name = parent.get("ref")
                        if parent_name in self.mName2Task:
                            parent_task = self.mName2Task[parent_name]
                            child_task.addParent(parent_task)
                            parent_task.addChild(child_task)
    #           if node.tag == "child":
    # pList = node.findall(".//Element")  # Assuming "getChildren()" gets all child elements
    # childName = node.get("ref")

    # if childName in mName2Task:
    #     childTask = mName2Task[childName]

    #     for parent in pList:
    #         parentName = parent.get("ref")

    #         if parentName in mName2Task:
    #             parentTask = mName2Task[parentName]
    #             parentTask.add_child(childTask)
    #             childTask.add_parent(parentTask)

            roots = []

            for task in self.mName2Task.values():
                task.setDepth(0)
                if not task.parentList:
                    roots.append(task)

            for root_task in roots:
                self.set_depth(root_task, 1)

            self.mName2Task.clear()

        except ET.ParseError:
            print("3")
            print("XML Parse Error; Please make sure your DAX file is valid")

        except IOError:
            print("4")
            print("IO Error; Please make sure the DAX path is correctly set in your config file")

        except Exception as e:
            print("5")
            print("Parsing Exception:", str(e))


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Environment

In [ ]:
import random
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
import gym
from gym import spaces

In [ ]:
a=WorkflowParser(1)
a.parse()

In [ ]:
#ENVIRONMENT
#Task- memory and PE? and how to implement Workflow
#FL implementation, course
# other model in DC 2
class Datacenter:
  def __init__(self, name, location, memory, cost_per_hour, cost_per_memory, bandwidth): #cost in Rs, memory in GBs, #Cost_per_memory strorage, BW,
    self.name = name
    self.location = location
    self.memory = memory
    self.cost_per_hour = cost_per_hour
    self.cost_per_memory = cost_per_memory
    self.bandwidth = bandwidth

  def add_host(self, host):
    self.hosts.append(host)

  def get_hosts(self):
    return self.hosts

  def show_hosts(self):
    print(self.hosts)
#Type List: MIPS,ID
PE_Type_list = {
    '1':
    {
        'MIPS': 100
    },
    '2':
    {
        'MIPS': 50
    },
    '3':
    {
        'MIPS': 10
    }
}

class Host:
  def __init__(self, name, Datacenter, cpu_cores, memory, ram, pe_type, bandwidth):#RAM, BW,Storage,PE ()Type List
    self.name = name
    self.Datacenter = Datacenter
    self.cpu_cores = cpu_cores
    self.memory = memory
    self.ram = ram
    self.pe_type = pe_type
    self.bandwidth = bandwidth
    self.vms = []

  def add_vm(self, vm):
    self.vms.append(vm)

  def get_mips(self, pe_type):
    return PE_Type_List[pe_type][0]

  def get_vms(self):
    return self.vms

VM_type = { #Task_Memory,
    'Type_1': {
          'Task_Memory' : 256,
          'mips' : 1000,
          'cost' : 0.10,
          'maxFreq' : 1.0,
          'minFreq' : 0.50,
          'minVoltage' : 5.0,
          'maxVoltage' : 7.0,
          'lambda' : 0.000150
    },
    'Type_2': {
          'Task_Memory' : 512,
          'mips' : 1500,
          'cost' : 0.20,
          'maxFreq' : 1.5,
          'minFreq' : 0.75,
          'minVoltage' : 7.0,
          'maxVoltage' : 9.0,
          'lambda' : 0.000100
        },
    'Type_3':
        {
          'Task_Memory' : 1024,
          'mips' : 2000,
          'cost' : 0.32,
          'maxFreq' : 2.0,
          'minFreq' : 1.00,
          'minVoltage' : 9.0,
          'maxVoltage' : 11.0,
          'lambda' : 0.000080
        },
    'Type_4':
        {
          'Task_Memory' : 500,
          'mips' : 2500,
          'cost' : 0.46,
          'maxFreq' : 2.5,
          'minFreq' : 1.25,
          'minVoltage' : 11.0,
          'maxVoltage' : 13.0,
          'lambda' : 0.000045
        },
    'Type_5':
        {
          'Task_Memory' : 560,
          'mips' : 3000,
          'cost' : 0.58,
          'maxFreq' : 3.0,
          'minFreq' : 1.50,
          'minVoltage' : 13,
          'maxVoltage' : 15,
          'lambda' : 0.000040
        },
    'Type_6':
        {
          'Task_Memory' : 250,
          'mips' : 3500,
          'cost' : 0.73,
          'maxFreq' : 3.5,
          'minFreq' : 1.75,
          'minVoltage' : 15,
          'maxVoltage' : 17,
          'lambda' : 0.000025
        },
    'Type_7':
        {
          'Task_Memory' : 150,
          'mips' : 4000,
          'cost' : 0.9,
          'maxFreq' : 4.0,
          'minFreq' : 2.00,
          'minVoltage' : 17,
          'maxVoltage' : 19,
          'lambda' : 0.000010
        },
    'Type_8':
        {
          'Task_Memory' : 750,
          'mips' : 4500,
          'cost' : 1.05,
          'maxFreq' : 4.5,
          'minFreq' : 2.25,
          'minVoltage' : 19,
          'maxVoltage' : 21,
          'lambda' : 0.000005
        },
    'Type_9':
        {
          'Task_Memory' : 2048,
          'mips' : 500,
          'cost' : 1.40,
          'maxFreq' : 5.5,
          'minFreq' : 2.75,
          'minVoltage' : 23,
          'maxVoltage' : 25,
          'lambda' : 0.000001
        }

}

class VM:
  def __init__(self, name, Host, Type):
    self.name = name
    self.Host = Host
    self.Type = Type
    # All_VMs.append(self.name)

# class Task:
#   def __init__(self, name, memory, length, PE):
#     self.name = name
#     self.length = length
#     self.memory = memory
#     self.PE = PE



def check_memory(datacenter, host_list):
    datacenter_memory=datacenter.memory
    host_memory=0
    for _ in host_list :
      host_memory += _.memory
    if host_memory > datacenter_memory :
      return True
    return False

def create_suitable_vms(task_memory, vm_memory):
    suitable_vms = []
    for t in task_memory:
        suitable_vms_for_task = []
        for v in range (len(vm_memory)):
            if t <= vm_memory[v]:
                suitable_vms_for_task.append(v)
        suitable_vms.append(suitable_vms_for_task)
    return suitable_vms


class VMAllocationEnv:
    def __init__(self, no_of_tasks, no_of_vms, task_list, task_lengths):
        super().__init__()
        self.task_lengths = task_lengths
        self.task_list = task_list
        self.no_of_tasks = no_of_tasks
        self.no_of_vms = no_of_vms
        self.observation_space = spaces.Box(low=0, high=1, shape=(no_of_vms, no_of_tasks), dtype=np.float32)
        # x,y = random.randint(0, len(action_space) - 1),random.randint(0, len(action_space[0]) - 1)
        self.state = np.zeros((no_of_vms, no_of_tasks), dtype = int)

    def action_space(self, task_id):
      x,y = task_id-1, (random.randint(0, no_of_vms-1))
      self.state[y][x]=1
      return [x,y]

    def calculate_reward(self, state, action, task_lengths):
        next_state = self.get_next_state(state, action)
        makespan = self.calculate_makespan(next_state, task_lengths)
        return 1.0/float(makespan)

    def get_next_state(self, state, action):
        vm_idx = action[1]
        task_idx = action[0]
        next_state = state
        # queue = []
        # queue.append(a.taskList)
        # for i in queue:
        #   if next_state[vm_idx][task.taskId-1]
        # print("n_s: ",next_state)
        # print("::", next_state[0])
        # print(":::", next_state[1])
        for i in range(len(next_state)):
            # print(next_state[i][task_idx])
            if (next_state[i][task_idx] == 1):
                next_state[i][task_idx] = 0
            next_state[vm_idx][task_idx] = 1
        return next_state

    def calculate_makespan(self, state, task_lengths):
        makespan = 0.0
        for i in range(len(state)):
            time_in_row = 0.0
            for j in range(len(state[0])):
                if (state[i][j] == 1):
                    time_in_row += task_lengths[j]
            makespan = max(makespan, time_in_row)
        # print("Makespan in env : ", makespan)
        return makespan

    def are_all_assigned(self, state): #true if f==no_of_tasks
        f = 0
        # print("State in step: ", state)
        for i in range(len(state[0])):
            for j in range(len(state)):
                if (state[j][i] == 1):
                    f += 1
        if (f >= self.no_of_tasks):
                # print("All are assigned!")
                return True
        return False

    def step(self, action):
        reward = self.calculate_reward(self.state, action, self.task_lengths)
        self.state = self.get_next_state(self.state, action)
        done = self.are_all_assigned(self.state)
        truncated = False
        info = {}

        # print("Step Called!")
        return self.state, reward, done, info

    def reset(self):
        self.state = np.zeros((self.no_of_vms, self.no_of_tasks), dtype = int)
#       self.state = self.state.flatten()
        # print(self.state)
        return self.state

D2 = Datacenter("D2", "Bangalore", 200000, 7.5, 7.5, 300) #name, location, memory, cost_per_hour, cost_per_memory, bandwidth

host_list = [
    Host("host1", D2, 2, 1024,2,1,10), #name, Datacenter, cpu_cores, memory, ram, pe_type, bandwidth
    Host("host2", D2, 4, 2048,4,2,25),
    Host("host3", D2, 8, 512,4,3,100),
    Host("host4", D2, 16, 4096,8,4,75)
]

if(check_memory(D2, host_list)):
  print("Memory limit Exceeded")

#vm_list: append vm type uniformly uniformly
vm_list = [
    VM("VM1", "host1", "Type_1"),
    VM("VM2", "host1", "Type_2"),
    VM("VM3", "host2", "Type_3"),
    VM("VM4", "host2", "Type_1"),
    VM("VM5", "host3", "Type_2"),
    VM("VM6", "host3", "Type_3"),
    VM("VM7", "host4", "Type_1"),
    VM("VM8", "host4", "Type_2"),
    VM("VM9", "host1", "Type_1"),
]


# task_list = [
#     Task("task1", 1024, 1000,1), #name, memory, length, PE
#     Task("task2", 512, 500,1),
#     Task("task3", 256, 250,1),
#     Task("task4", 128, 100,1),
#     Task("task5", 124, 190,1),
#     Task("task6", 500, 10,1),
#     Task("task7", 256, 20,1),
#     Task("task8", 128, 130,1),
#     Task("task9", 1024, 240,1),
#     Task("task10", 512, 550,1),
#     Task("task11", 256, 210,1),
#     Task("task12", 128, 690,1),
#     Task("task13", 156, 110,1),
#     Task("task14", 128, 620,1)
# ]

# task_memory = []
# vm_memory = []
# for _ in vm_list:
#   vm_memory.append(VM_type[_.Type]['Task_Memory'])
# for _ in task_list:
#   task_memory.append(_.memory)

# print("VM Memory :", vm_memory)
# print("Task Memory :", task_memory)

# suitable_vms = create_suitable_vms(task_memory, vm_memory)
# print("Suitable VMs:" , suitable_vms)

task_lengths = []
for _ in a.taskList:
  task_lengths.append(_.taskLength)

# vm_available = [0] * len(vm_list)

# def allocate_task(task, vms, task_memory, vm_memory):
#     # suitable_vms = create_suitable_vms(task_memory, vm_memory)
#     vm = None
#     # Randomly select a VM from the suitable VMs
#     index = random.randint(0, len(suitable_vms[task]) - 1)
#     vm_index = suitable_vms[task][index]
#     if(vm_available[vm_index]==0):
#       vm = vm_index
#       vm_available[vm_index]=1
#     # If the task cannot be allocated to any of the suitable VMs, print the message "Unallocated task"
#     if vm is None:
#         print("Unallocated task :", task)
#         return None,None
#     return vm,vm_index

# action_space = array = [[0 for i in range(len(vm_list))] for j in range(len(task_list))]
# print(action_space)

# def allocate_tasks(tasks, vms, task_memory, vm_memory):
#     allocated_tasks = []
#     for task in range(len(tasks)):
#         vm, vm_index = allocate_task(task, vms, task_memory, vm_memory)
#         print(vm_index)
#         if vm and vm_index is not None:
#             allocated_tasks.append((tasks[task].memory, vm))
#             print(task,vm_index,len(action_space))
#             print(len(action_space[task]))
#             action_space[task][vm_index]=1
#     return allocated_tasks

# res = allocate_tasks(task_list, vm_list, task_list, task_memory, vm_memory)
# print(res)
# print(action_space)
# create action_space


'''
  for every task, find out all suitable vms, create list of suitable vms, allocate to any one of them randomly
  keep a list of vm occupied and vm time score.
  while allocating tasks randomly to suitable vms, check it's availibility, if it's unavailable, allocate the task to some other random location
  if all of the locations are unavailable, return unallocated message. Do this repeatedly until makespan is minimized.
  OR KEEP ALLOCATING VMS ONCE A BATCH.
'''

# print("Task_List :", task_lengths)
no_of_vms = len(vm_list)
no_of_tasks = len(task_lengths)

# env = VMAllocationEnv(vm_list, task_list)
env = VMAllocationEnv(no_of_tasks, no_of_vms, a.taskList, task_lengths)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class QNet(nn.Module):
    def __init__(self, hidden_dim=64):
        super().__init__()

        self.hidden = nn.Linear(no_of_vms*no_of_tasks+2, hidden_dim)
        self.output = nn.Linear(hidden_dim, 2)

    def forward(self, s, a):
#         print("QNetwork outs: s: ",s," a : ",  a)
        a = np.squeeze(a)
        s = np.squeeze(s)
        outs = torch.concat((torch.flatten(s[0]),a[0]), dim=-1)
        for t in range(1,len(s)):
            x = torch.flatten(s[t])
            output=torch.concat((x, a[t]), dim=-1)
            outs=torch.concat((outs,output), dim=-1)
        outs = torch.reshape(outs,(len(s), 1, (no_of_vms*no_of_tasks)+len(a[0])))
        outs = self.hidden(outs)
        outs = F.relu(outs)
        outs = self.output(outs)
        return outs

q_origin_model = QNet().to(device)  # Q_phi
q_target_model = QNet().to(device)  # Q_phi'
_ = q_target_model.requires_grad_(False)  # target model doen't need grad

In [ ]:
class PolicyNet(nn.Module):
    def __init__(self, hidden_dim=32):
        super().__init__()

        self.hidden = nn.Linear((no_of_vms*no_of_tasks), hidden_dim)
        self.output = nn.Linear(hidden_dim, 2)

    def forward(self, s):
        outs = torch.flatten(s[0])
        for t in range(1,len(s)):
            x = torch.flatten(s[t])
#             output=torch.concat((x, a[t]), dim=-1)
            outs=torch.concat((outs,x), dim=-1)
        outs.resize_([len(s),(no_of_vms*no_of_tasks)])
        s = outs
        outs = self.hidden(s)
        outs = F.relu(outs)
        outs = self.output(outs)
        outs = torch.tanh(outs)
        return outs

mu_origin_model = PolicyNet().to(device)  # mu_theta
mu_target_model = PolicyNet().to(device)  # mu_theta'
_ = mu_target_model.requires_grad_(False)  # target model doen't need grad

In [ ]:
gamma = 0.80 #0.99
opt_q = torch.optim.AdamW(q_origin_model.parameters(), lr=0.05)
opt_mu = torch.optim.AdamW(mu_origin_model.parameters(), lr=0.05)

def optimize(states, actions, rewards, next_states, dones):
    # Convert to tensor
    # print("states : ", states)
    # print("actions : ", actions)
    states = torch.tensor(states, dtype=torch.float).to(device)
    actions = torch.tensor(actions, dtype=torch.float).to(device)
    actions = actions.unsqueeze(dim=1)
    rewards = torch.tensor(rewards, dtype=torch.float).to(device)
    rewards = rewards.unsqueeze(dim=1)
    next_states = torch.tensor(next_states, dtype=torch.float).to(device)
    dones = torch.tensor(dones, dtype=torch.float).to(device)
    dones = dones.unsqueeze(dim=1)

    # Optimize critic loss
    opt_q.zero_grad()
    q_org = q_origin_model(states, actions)
    mu_tgt_next = mu_target_model(next_states)
    q_tgt_next = q_target_model(next_states, mu_tgt_next)
    q_tgt = rewards + gamma * (1.0 - dones) * q_tgt_next
    # print(q_org.size(), q_tgt.size())
    loss_q = F.mse_loss(q_org, q_tgt, reduction="none")
    loss_q.sum().backward()
    opt_q.step()

    # Optimize actor loss
    opt_mu.zero_grad()
    mu_org = mu_origin_model(states)
    for p in q_origin_model.parameters():
        p.requires_grad = False # disable grad in q_origin_model before computation
    q_tgt_max = q_origin_model(states, mu_org)
    (-q_tgt_max).sum().backward()
    opt_mu.step()
    for p in q_origin_model.parameters():
        p.requires_grad = True # enable grad again

In [ ]:
tau = 0.001 #0.002

def update_target():
    for var, var_target in zip(q_origin_model.parameters(), q_target_model.parameters()):
        var_target.data = tau * var.data + (1.0 - tau) * var_target.data
    for var, var_target in zip(mu_origin_model.parameters(), mu_target_model.parameters()):
        var_target.data = tau * var.data + (1.0 - tau) * var_target.data

In [ ]:
class replayBuffer:
    def __init__(self, buffer_size: int):
        self.buffer_size = buffer_size
        self.buffer = []
        self._next_idx = 0

    def add(self, item):
        if len(self.buffer) > self._next_idx:
            self.buffer[self._next_idx] = item
        else:
            self.buffer.append(item)
        if self._next_idx == self.buffer_size - 1:
            self._next_idx = 0
        else:
            self._next_idx = self._next_idx + 1
#         print("Item_to_be_added : ",item)
#         print("Buffer", self.buffer)

    def sample(self, batch_size):
        indices = [random.randint(0, len(self.buffer) - 1) for _ in range(batch_size)]
        states   = [self.buffer[i][0] for i in indices]
        actions  = [self.buffer[i][1] for i in indices]
        rewards  = [self.buffer[i][2] for i in indices]
        n_states = [self.buffer[i][3] for i in indices]
        dones    = [self.buffer[i][4] for i in indices]
#         print("Buffer :", [states, actions, rewards, n_states, dones] )
        return states, actions, rewards, n_states, dones

    def length(self):
        return len(self.buffer)

buffer = replayBuffer(buffer_size=2000)

In [ ]:
a=WorkflowParser(1)
a.parse()

In [ ]:
#Keep similar lr
from tqdm import tqdm
batch_size = 10 #250
from queue import Queue
reward_records = []
episodes = []
final_makespan = []
for i in tqdm(range(1000)):   #2000
    # Run episode till done
    s = env.reset()
    # print("S : ",s)
    cum_reward = 0
    makespans = []
    q = Queue()
    # queue.append(a.taskList)
    for _ in a.taskList:
      q.put(_)

    while q.qsize()>0:
      # print("Queue: ", q)
      x = q.get()
      flag = False
      for j in x.parentList:
        if j.allocated==False:
          q.put(x)
          flag = True
          break
      if flag==True:
        continue
      else:
        x.allocated = True
        act = env.action_space(x.taskId)
        # print("Sample Action: ", a)
        s_next, r, done, _ = env.step(act)
#         print("action : ", a)
        buffer.add([s, act, r, s_next, float(done)])
        # print("Buffer: ", buffer)
        if done:
          makespans.append(1/r)
          print("Makespans: ", makespans)
          cum_reward += r
        # Train (optimize parameters)
        if buffer.length() >= batch_size:
            states, actions, rewards, n_states, dones = buffer.sample(batch_size)
            # print("States :", states )
            # print("actions :", actions )
            # print("rewards :", rewards )
            # print("n_states :", n_states )
            # print("dones :", dones)
            optimize(states, actions, rewards, n_states, dones)
            update_target()
        s = s_next


#     while not done:
#         # a = pick_sample(s)
# #         print("a : ",a)
#         if all_parent_done:
#         a = env.action_space.sample()
#         # print("Sample Action: ", a)
#         s_next, r, done, _ = env.step(a)
# #         print("action : ", a)
#         buffer.add([s, a, r, s_next, float(done)])
#         if done:
#           makespans.append(1/r)
#           cum_reward += r
#         # Train (optimize parameters)
#         if buffer.length() >= batch_size:
#             states, actions, rewards, n_states, dones = buffer.sample(batch_size)
#             # print("States :", states )
#             # print("actions :", actions )
#             # print("rewards :", rewards )
#             # print("n_states :", n_states )
#             # print("dones :", dones)
#             optimize(states, actions, rewards, n_states, dones)
#             update_target()
#         s = s_next

    # Output total rewards in episode (max 500)
    episodes.append(i)
    # print("Episode :", i)
    print("Run episode ",i, " with rewards ", cum_reward)

    # print("Makespan :", np.average(makespans))
    reward_records.append(cum_reward)
    final_makespan.append(min(makespans))
    # stop if reward mean > 100.0
    if np.average(reward_records[-50:]) > 100.0:
        break

print("Final Makespan : ", min(final_makespan))
print("\n Done")

  0%|          | 0/1000 [00:00<?, ?it/s]<ipython-input-221-aab6093cba17>:25: UserWarning: Using a target size (torch.Size([10, 10, 2])) that is different to the input size (torch.Size([10, 1, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss_q = F.mse_loss(q_org, q_tgt, reduction="none")
  0%|          | 1/1000 [00:00<12:17,  1.35it/s]

Makespans:  [139160.0]
Run episode  0  with rewards  7.185972980741593e-06


  0%|          | 2/1000 [00:01<12:58,  1.28it/s]

Makespans:  [151060.0]
Run episode  1  with rewards  6.619886137958427e-06


  0%|          | 3/1000 [00:02<13:04,  1.27it/s]

...

Makespans:  [195630.0]
Run episode  997  with rewards  5.111690436027194e-06


100%|█████████▉| 999/1000 [19:30<00:01,  1.34s/it]

Makespans:  [175080.0]
Run episode  998  with rewards  5.711674663011195e-06


100%|██████████| 1000/1000 [19:31<00:00,  1.17s/it]

Makespans:  [160130.0]
Run episode  999  with rewards  6.244925997626928e-06
Final Makespan :  130680.0

 Done
